# Soluciones Proyecto ETL

In [1]:
import pandas as pd
import requests
import datetime

## Ejercicio 1

In [2]:
postal_codes_raw = pd.read_csv('./ETL_project-postal_codes.csv', sep=';', encoding = "utf-8")

In [4]:
postal_codes_raw.iloc[2:4]

,provincia,poblacion,Código Postal
2,Araba/Álava,Vitoria-Gasteiz,1001
3,Araba/Álava,Vitoria-Gasteiz,1002


In [5]:
postal_codes_raw.groupby('provincia').nunique().head(3)

,poblacion,Código Postal
provincia,,
A Coru?a,93,382
Albacete,87,147
Alicante/Alacant,141,204


In [6]:
data = []
ids = 1
for i in postal_codes_raw.index:
    row = postal_codes_raw.iloc[i]
    
    result = {"id": ids + 1000, "cp": row['Código Postal']}
    ids += 1
    
    prov_names = row['provincia'].replace('?', 'ñ').split('/')
    city_names = row['poblacion'].replace('?', 'ñ').split('/')
    
    if len(prov_names) == 1:
        result['provincia'] = prov_names[0]
        result['provincia_local'] = prov_names[0]
    else:
        result['provincia'] = prov_names[1]
        result['provincia_local'] = prov_names[0]
        
    if len(city_names) == 1:
        result['ciudad'] = city_names[0]
        result['ciudad_local'] = city_names[0]
    else:
        result['ciudad'] = city_names[1]
        result['ciudad_local'] = city_names[0]
        
    data.append(result)
    
cp_table = pd.DataFrame(data, columns= data[0].keys())
    

In [7]:
cp_table.iloc[2:4]

,id,cp,provincia,provincia_local,ciudad,ciudad_local
2,1003,1001,Álava,Araba,Vitoria-Gasteiz,Vitoria-Gasteiz
3,1004,1002,Álava,Araba,Vitoria-Gasteiz,Vitoria-Gasteiz


## Ejercicio 2

In [8]:
basic_headers = {'User-Agent': 'Mozilla/5.0'}
endpoint = 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM'
payload = requests.get(endpoint, headers=basic_headers)

In [9]:
population_data = payload.json()

In [10]:
population_clean = []
pop_err = []
for reg in population_data:
    
    try:
        if reg['MetaData'][1]['Variable']['Codigo'] == 'MUN':
            temp = {"id": reg['COD']}
            temp['Poblacion'] = reg['MetaData'][1]['Nombre']
            temp['Origen'] = reg['MetaData'][2]['Nombre']

            for row in reg['Data']:
                temp[datetime.datetime.strptime(row['Fecha'], '%Y-%m-%dT%H:%M:%S.000+01:00').year] = int(row['Valor'])

            population_clean.append(temp)
    
    except Exception as e:
            pop_err.append(reg)
            print(e)

In [11]:
pop_err

[]

In [12]:
len(population_clean)

26520

In [13]:
population_std = pd.DataFrame(population_clean, columns=population_clean[0].keys())
population_std.head()

,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5662094,Agullana,Total,903.0,885.0,863.0,831.0,841.0,831.0,841.0,...,858.0,839.0,840.0,812.0,806.0,753.0,756.0,759.0,746.0,723.0
1,PC5662093,Agullana,España,735.0,711.0,704.0,689.0,697.0,692.0,704.0,...,678.0,657.0,651.0,645.0,648.0,645.0,662.0,670.0,657.0,664.0
2,PC5662092,Agullana,Extranjero,168.0,174.0,159.0,142.0,144.0,139.0,137.0,...,180.0,182.0,189.0,167.0,158.0,108.0,94.0,89.0,89.0,59.0
3,PC5662063,Agullana,Europa (sin España),71.0,79.0,67.0,56.0,57.0,53.0,55.0,...,94.0,103.0,103.0,92.0,84.0,59.0,51.0,55.0,53.0,33.0
4,PC5662062,Agullana,UE28 sin España,NaN,NaN,57.0,50.0,54.0,49.0,51.0,...,89.0,97.0,96.0,87.0,79.0,56.0,48.0,49.0,46.0,28.0


In [14]:
duplicates = population_std['id'].duplicated()
duplicated_df = population_std[duplicates]
duplicated_df

,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003


## Ejercicio 3

In [15]:
df = pd.read_csv('datosgobes.csv', encoding = "utf-8")

In [16]:
# df.to_excel('datosgobes.xlsx', index=False)

In [17]:
df['DISTRIBUCIONES'][0:3]

0    [TITLE_es]pdf[ACCESS_URL]https://www.aepd.es/r...
1    [ACCESS_URL]https://www.aepd.es/informes/[MEDI...
2    [TITLE_es]Códigos y otros libros electrónicos[...
Name: DISTRIBUCIONES, dtype: object

This is our row of interest. The one used in the previous excecise:

In [18]:
my_row = df[df['DISTRIBUCIONES'].str.contains('https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM', na=False, regex=False)]
my_row

,URL,IDENTIFICADOR,TÍTULO,DESCRIPCIÓN,TEMÁTICAS,ETIQUETAS,FECHA DE CREACIÓN,FECHA DE ÚLTIMA MODIFICACIÓN,FRECUENCIA DE ACTUALIZACIÓN,IDIOMAS,ÓRGANO PUBLICADOR,CONDICIONES DE USO,COBERTURA GEOGRÁFICA,COBERTURA TEMPORAL,VIGENCIA DEL RECURSO,RECURSOS RELACIONADOS,NORMATIVA,DISTRIBUCIONES
52701,https://datos.gob.es/catalogo/b62c72eb-b291-40...,urn:ine:es:TABLA:T3:65:33791,"[en]Population by sex, municipalities and coun...","[en]Table of INEBase Population by sex, munici...",Demografía,[en]Abroad//Africa//Algeria//America//Brazil//...,2012-01-13T12:00:00+0100,2022-01-17T12:00:00+0100,[TYPE]http://www.w3.org/2006/time#years[VALUE]1,es//en,Instituto Nacional de Estadística,https://www.ine.es/aviso_legal,Girona,2003-01-01T01:00:00+0100-2022-01-01T01:00:00+0100,NaN,https://www.ine.es/dyngs/IOE/es/operacion.htm?...,https://www.ine.es/dynt3/metadatos/es/Respuest...,[TITLE_en]Html[TITLE_es]Html[ACCESS_URL]https:...


In [19]:
pd.set_option('display.max_colwidth', None)  # Set the option to display the full content of each cell
my_row['DISTRIBUCIONES']

52701    [TITLE_en]Html[TITLE_es]Html[ACCESS_URL]https://www.ine.es/jaxiT3/Tabla.htm?t=33791[MEDIA_TYPE]HTML//[TITLE_en]Excel: XLSx extension[TITLE_es]Excel: Extension XLSx[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/xlsx/33791.xlsx[MEDIA_TYPE]XLSX//[TITLE_en]CSV: Separated by ;[TITLE_es]CSV: separado por ;[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/csv_bdsc/33791.csv[MEDIA_TYPE]CSV//[TITLE_en]CSV: Tab Separated[TITLE_es]CSV: separado por tabuladores[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/csv_bd/33791.csv[MEDIA_TYPE]CSV//[TITLE_en]Json[TITLE_es]Json[ACCESS_URL]https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM[MEDIA_TYPE]JSON//[TITLE_en]Pc-Axis[TITLE_es]Pc-Axis[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/px/33791.px[MEDIA_TYPE]PC-Axis//[TITLE_en]Excel: XLS extension[TITLE_es]Excel: Extensión XLS[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/xls/33791.xls[MEDIA_TYPE]XLS//[TITLE_en]Plain Text: separated by comma[TITLE_es]Texto plano: separado por ,[ACCESS_URL]https

I still have my row identified ;)

Let's see its characteristics:

In [20]:
pd.reset_option('display.max_colwidth')
df.iloc[52701]['TÍTULO']

'[en]Population by sex, municipalities and country of birth (API identifier: 33791)[es]Población por sexo, municipios y país de nacimiento (Identificador API: 33791)'

Let us reduce the dataframe to the rows of interest:

In [21]:
filtered_df = df[df['TÍTULO'].str.contains("[en]Population by sex, municipalities and country of birth", case=False, regex=False)]
print(len(filtered_df))

53


In [22]:
# filtered_df.to_csv('Data_of_interest.csv', index=False)
# filtered_df.to_excel('Data_of_interest.xlsx', index=False)

Let us extract URLs:

In [23]:
for i in filtered_df.index:
    if i==52701:
        print(filtered_df.loc[i].to_dict()['DISTRIBUCIONES'])
        dist = filtered_df.loc[i].to_dict()['DISTRIBUCIONES'].split('[TITLE_en]')

        for resource in dist:

            if resource[:4] == 'Json':
                print()
                print(resource)
                link = resource.split('[ACCESS_URL]')[1].split('[MEDIA_TYPE]')[0]
                print()
                print('Link -->',link)
                mylink = link



[TITLE_en]Html[TITLE_es]Html[ACCESS_URL]https://www.ine.es/jaxiT3/Tabla.htm?t=33791[MEDIA_TYPE]HTML//[TITLE_en]Excel: XLSx extension[TITLE_es]Excel: Extension XLSx[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/xlsx/33791.xlsx[MEDIA_TYPE]XLSX//[TITLE_en]CSV: Separated by ;[TITLE_es]CSV: separado por ;[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/csv_bdsc/33791.csv[MEDIA_TYPE]CSV//[TITLE_en]CSV: Tab Separated[TITLE_es]CSV: separado por tabuladores[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/csv_bd/33791.csv[MEDIA_TYPE]CSV//[TITLE_en]Json[TITLE_es]Json[ACCESS_URL]https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM[MEDIA_TYPE]JSON//[TITLE_en]Pc-Axis[TITLE_es]Pc-Axis[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/px/33791.px[MEDIA_TYPE]PC-Axis//[TITLE_en]Excel: XLS extension[TITLE_es]Excel: Extensión XLS[ACCESS_URL]https://www.ine.es/jaxiT3/files/t/xls/33791.xls[MEDIA_TYPE]XLS//[TITLE_en]Plain Text: separated by comma[TITLE_es]Texto plano: separado por ,[ACCESS_URL]https://www.in

Let us proceed with all the rows:

In [25]:
distribuciones = []
n = 0

for i in filtered_df.index:
    dist = filtered_df.loc[i].to_dict()['DISTRIBUCIONES'].split('[TITLE_en]')
    
    for resource in dist:
        if resource[:4] == 'Json':
            link = resource.split('[ACCESS_URL]')[1].split('[MEDIA_TYPE]')[0]
            if link == mylink:
                print('-- The following is my link:')

                print(f'Link position: {n} -->',link)
            n += 1

            distribuciones.append(link)


-- The following is my link:
Link position: 38 --> https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33791?tip=AM


In [26]:
print(len(distribuciones))
my_distributions = distribuciones[0:3]
my_distributions

53


['https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33923?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33827?tip=AM',
 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/33809?tip=AM']

In [27]:
def extract_data(my_distributions):
    basic_headers = {'User-Agent': 'Mozilla/5.0'}
    population_all = []
    n = 0
    for link in my_distributions:
        print('Number of links processed:', n)
        n += 1
        
        payload = requests.get(link, headers=basic_headers)
        payload_json = payload.json()
        
        for reg in payload_json:

            try:
                if reg['MetaData'][1]['Variable']['Codigo'] == 'MUN':
                    temp = {"id": reg['COD']}

                    temp['Poblacion'] = reg['MetaData'][1]['Nombre']
                    temp['Origen'] = reg['MetaData'][2]['Nombre']

                    for row in reg['Data']:
                        temp[datetime.datetime.strptime(row['Fecha'], '%Y-%m-%dT%H:%M:%S.000+01:00').year] = int(row['Valor'])

                    population_all.append(temp)
            except Exception as e:
                print(e)
                pass
    
    return population_all

In [28]:
population_all = extract_data(distribuciones[0:2])

population_std = pd.DataFrame(population_all, columns=population_all[0].keys())
print(len(population_std))
population_std.head()

Number of links processed: 0
Number of links processed: 1
47280


,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5664906,Abejar,Total,273.0,273.0,286.0,297.0,302.0,318.0,327.0,...,367.0,378.0,380.0,383.0,390.0,405.0,406.0,392.0,399.0,395.0
1,PC5664905,Abejar,España,242.0,239.0,257.0,264.0,269.0,290.0,300.0,...,333.0,340.0,337.0,344.0,350.0,373.0,379.0,376.0,386.0,383.0
2,PC5664904,Abejar,Extranjero,31.0,34.0,29.0,33.0,33.0,28.0,27.0,...,34.0,38.0,43.0,39.0,40.0,32.0,27.0,16.0,13.0,12.0
3,PC5664875,Abejar,Europa (sin España),12.0,15.0,17.0,20.0,20.0,15.0,16.0,...,17.0,18.0,18.0,22.0,19.0,13.0,9.0,6.0,2.0,1.0
4,PC5664874,Abejar,UE28 sin España,NaN,NaN,16.0,19.0,19.0,14.0,15.0,...,16.0,17.0,17.0,21.0,18.0,13.0,9.0,6.0,2.0,1.0


In [30]:
population_all = extract_data(distribuciones[2:3])
population_df = pd.DataFrame(population_all, columns=population_all[0].keys())
print(len(population_df))
population_df.head()

Number of links processed: 0
9600


,id,Poblacion,Origen,2022,2021,2020,2019,2018,2017,2016,...,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003
0,PC5660878,Alájar,Total,802.0,814.0,759.0,756.0,763.0,777.0,814.0,...,803.0,798.0,806.0,808.0,802.0,811.0,821.0,771.0,750.0,769.0
1,PC5660877,Alájar,España,732.0,740.0,697.0,696.0,707.0,720.0,753.0,...,739.0,738.0,747.0,750.0,743.0,749.0,757.0,729.0,713.0,724.0
2,PC5660876,Alájar,Extranjero,70.0,74.0,62.0,60.0,56.0,57.0,61.0,...,64.0,60.0,59.0,58.0,59.0,62.0,64.0,42.0,37.0,45.0
3,PC5660847,Alájar,Europa (sin España),41.0,47.0,40.0,41.0,38.0,39.0,44.0,...,52.0,50.0,49.0,46.0,49.0,51.0,53.0,36.0,33.0,41.0
4,PC5660846,Alájar,UE28 sin España,NaN,NaN,38.0,38.0,35.0,36.0,39.0,...,48.0,46.0,46.0,43.0,46.0,47.0,52.0,35.0,31.0,39.0
